In [14]:
df = pd.read_csv('c:/Users/duPar/Downloads/서울시버스정류소좌표데이터(2022.08.24).csv',encoding='CP949')
df.columns = ['node_id','ars_id','station_nm','x_axis','y_axis']
df['base_ym'] = datetime.today().strftime("%Y%m")
df.to_csv('c:/Users/duPar/Downloads/서울시버스정류소좌표데이터(2022.08.24)_최종.csv',index=False,encoding='utf-8-sig')

In [24]:
df

,node_id,ars_id,station_nm,x_axis,y_axis,base_ym
0,100000001,1001,종로2가사거리,126.987786,37.569764,202210
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179,202210
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709,202210
3,100000004,1004,종로2가.삼일교,126.987507,37.568582,202210
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230,202210
...,...,...,...,...,...,...
11162,124000334,25995,우성아파트,127.139326,37.550442,202210
11163,124000333,25996,우성아파트,127.140052,37.550620,202210
11164,124000332,25997,조일약국,127.123477,37.533686,202210
11165,124000331,25998,성내시장,127.125410,37.536199,202210


### 전국 버스
https://www.data.go.kr/data/15067528/fileData.do

In [14]:
### 0. import
import os
import numpy  as np
import pandas as pd
from datetime import datetime

# 경고 처리
import warnings
warnings.filterwarnings('ignore')


### 1. 파일 전처리
# 파일 불러오기
df = pd.read_csv("C:\oasis\git\data\업무 완료\전국 지하철, 버스 (역좌표, 승하차수)\한국교통안전공단_전국_버스정류장_좌표\전국버스정류장 위치정보.csv", encoding='CP949')
df.columns = ['node_id','node_nm','node_tp','center_yn','node_enm','lon','lat','work_datetime','node_mobile_id','sido_cd','sido_nm']

# 토산삼거리 널값 채우기
idx = df[df['station_nm']=='토산삼거리'].index
for i in idx:
    df['local_nm'][i] = '삼척시'

# 경위도 str으로 타입 변경
df['lat'] = df['lat'].astype(str)
df['lon'] = df['lon'].astype(str)

# 경위도 포인트로 묶기
df['point'] = np.nan
for i in df.index:
    df['point'][i] = 'POINT (' + df['lat'][i] + ' ' + df['lon'][i] + ')'
    
# polygon str 변환
df['point'] = df['point'].astype(str)
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB 연동
import psycopg2
import datetime

table = 'm1.bus_xy' # 테이블 지정

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,ST_GeomFromText(%s, 4326),%s)'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-20 09:44:18.261733  :  0
2022-10-20 09:44:22.292416  :  10000
2022-10-20 09:44:26.919847  :  20000
2022-10-20 09:44:30.788787  :  30000
2022-10-20 09:44:48.479630  :  40000
2022-10-20 09:45:00.985319  :  50000
2022-10-20 09:45:10.207222  :  60000
2022-10-20 09:45:24.154692  :  70000
2022-10-20 09:45:27.566992  :  80000
2022-10-20 09:45:31.366169  :  90000
2022-10-20 09:45:35.167779  :  100000
2022-10-20 09:45:47.739898  :  110000
2022-10-20 09:45:52.426892  :  120000
2022-10-20 09:45:56.292599  :  130000
2022-10-20 09:45:59.533701  :  140000
2022-10-20 09:46:02.958547  :  150000
2022-10-20 09:46:07.875333  :  160000
2022-10-20 09:46:17.400041  :  170000
2022-10-20 09:46:22.849278  :  180000
187645
